In [1]:
from model import MLP

model = MLP()

model

MLP(
  (l1): Linear(in_features=256, out_features=512, bias=True)
  (l2): LinW(in_features=512, out_features=512, bias=True)
  (l3): LinW(in_features=512, out_features=512, bias=True)
  (l4): Linear(in_features=512, out_features=10, bias=True)
  (gelu): GELU(approximate='none')
)